# Onboarding as a Data Scientist

This notebook is the Data Scientist view of part 5. You should have already run through steps 1-7 in the OM Authority notebook 5. If you have not done so proceed to port 8888 (either localhost or aws) and complete the steps in notebook `5. Onboarding a Data Scientist` first.



## 8. Initialising the Data Scientist Controller

This is very similar to the code used for the OM Authority accept the arguments used to configure the AriesAgentController are different.

The point to the ports and webhook location used when starting the data-scientist-agent ACA-Py instance. You can see these arguments in the `manage` script line 145.


In [1]:
%autoawait
import time
import asyncio
from termcolor import colored,cprint

from aries_basic_controller.aries_controller import AriesAgentController
import os

api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")


# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=admin_url, api_key=api_key)

# The location the controller spins up a service and listens for webhooks from the agent
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

agent_controller.init_webhook_server(webhook_host=webhook_host, webhook_port=webhook_port)

IPython autoawait is `on`, and set to use `asyncio`


## 9. Start Webhook Server and Register Listeners

We register listeners for the connections and issue-credential topic as before. However, the output will be slightly different because the Data Scientist is on the other side of the protocol to the OM Authority. We could not see any webhooks from this side before because they were managed by the mobile application you used.

In [2]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())


def cred_handler(payload):
    print("Handle Credential Webhook Payload")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange ID {exchange_id}")
    print("Agent Protocol Role", role)
    print("Protocol State ", state )
    if state == "offer_recieved" and role == "holder":
        print(f"You are being offered a credential with the following: {attributes}")
    if state == "credential_acked":
        print(colored("Your agent has stored the credential in it's wallet", "green", attrs=["bold"]))
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}

def connections_handler(payload):
    print("Handle Connection Webhook Payload")
    connection_id = payload["connection_id"]
    print("Connection ID", connection_id)

    state = payload['state']
    print("State", state)
    if state == 'active':
#         print('Connection {0} changed state to active'.format(connection_id))
        print(colored("Connection {0} changed state to active".format(connection_id), "red", attrs=["bold"]))


connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

agent_controller.register_listeners([cred_listener,connection_listener], defaults=True)

Subscribing too: issue_credential
Subscribing too: connections


## 10. Copy Invitation from OM Authority

You should have **copied** the invitation object output from step 7. in the OM Authority notebook. **First** paste that over the example object shown below, **then** run the cell.

In [5]:
invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '3c8aadda-3405-4ef9-9968-92c14f016230', 'label': 'OpenMined', 'recipientKeys': ['CJhCVsrLAArEzjKqrbKnP2fquJ7aFCrzkRDQaYWhc9e1'], 'serviceEndpoint': 'https://9b18-86-18-68-143.ngrok.io'}

## 11. Receive Invitation

Pay attention to the webhook messages that get output as the datascientist-agent exchanges messages with the om-authority. First messages are from the connections topic handler and then the issue-credential handler.

In this instance the Data Scientist agent is configured to accept any credentials it gets offered through a flag passed into the `aca-py start` command.

The flags set on all agents in this notebook series are in their respective .env files

They are:

* ACAPY_AUTO_ACCEPT_INVITES=true
* ACAPY_AUTO_ACCEPT_REQUESTS=true
* ACAPY_AUTO_PING_CONNECTION=true
* ACAPY_AUTO_RESPOND_MESSAGES=true
* ACAPY_AUTO_RESPOND_CREDENTIAL_PROPOSAL=true
* ACAPY_AUTO_RESPOND_CREDENTIAL_OFFER=true
* ACAPY_AUTO_RESPOND_CREDENTIAL_REQUEST=true
* ACAPY_AUTO_RESPOND_PRESENTATION_PROPOSAL=true
* ACAPY_AUTO_RESPOND_PRESENTATION_REQUEST=true
* ACAPY_AUTO_STORE_CREDENTIAL=true
* ACAPY_AUTO_VERIFY_PRESENTATION=true
* ACAPY_PRESERVE_EXCHANGE_RECORDS=true



In [7]:
await agent_controller.connections.receive_invitation(invitation)

Handle Connection Webhook Payload
Connection ID 17e911e0-9061-491a-a9c3-34078b4c96de
State invitation
Handle Connection Webhook Payload
Connection ID 17e911e0-9061-491a-a9c3-34078b4c96de
State request


{'my_did': 'BPJLH3NttWGPFw2WLdkn6b',
 'routing_state': 'none',
 'updated_at': '2021-10-08 11:48:01.101170Z',
 'invitation_key': 'CJhCVsrLAArEzjKqrbKnP2fquJ7aFCrzkRDQaYWhc9e1',
 'invitation_mode': 'once',
 'created_at': '2021-10-08 11:48:01.085715Z',
 'accept': 'auto',
 'state': 'request',
 'their_label': 'OpenMined',
 'request_id': 'e144c3db-174d-4a0f-9364-eeaa8368c80d',
 'connection_id': '17e911e0-9061-491a-a9c3-34078b4c96de',
 'their_role': 'inviter',
 'rfc23_state': 'request-sent'}

Handle Connection Webhook Payload
Connection ID 17e911e0-9061-491a-a9c3-34078b4c96de
State response
Handle Connection Webhook Payload
Connection ID 17e911e0-9061-491a-a9c3-34078b4c96de
State active
Connection 17e911e0-9061-491a-a9c3-34078b4c96de changed state to active
Handle Credential Webhook Payload
Credential exchange ID cc305207-9c17-41ea-b757-63d6a950207e
Agent Protocol Role holder
Protocol State  offer_received
Handle Credential Webhook Payload
Credential exchange ID cc305207-9c17-41ea-b757-63d6a950207e
Agent Protocol Role holder
Protocol State  request_sent
Handle Credential Webhook Payload
Credential exchange ID cc305207-9c17-41ea-b757-63d6a950207e
Agent Protocol Role holder
Protocol State  credential_received
Handle Credential Webhook Payload
Credential exchange ID cc305207-9c17-41ea-b757-63d6a950207e
Agent Protocol Role holder
Protocol State  credential_acked
Your agent has stored the credential in it's wallet


## 12. Review the output from the webhook listeners in the OM Authority notebook

The state from the issue-credential webhook payloads should be slightly different.

Don't forget to terminate the AriesAgentController in this tutorial

In [8]:
await agent_controller.terminate()

## Continue to Notebook 6.

Remember to start in the OM Authority jupyter notebook. We will not be interacting with the data scientist for this set of notebooks.